In [ ]:
print("Hello")

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, filtfilt, resample_poly
import matplotlib.pyplot as plt

# --- Step 1: Load CSV file ---
df = pd.read_csv('combined_patient_ecg_outcome.csv')

# --- Step 2: Parsing function for space-separated ECG string to 1D numpy array ---

def string_to_float_array_fixed(s):
    values = []
    for val in s.strip().split():
        try:
            values.append(float(val))
        except:
            values.append(np.nan)
    return np.array(values)

df['data_array'] = df['data'].apply(string_to_float_array_fixed)

# --- Step 3: NaN interpolation helper ---

def interpolate_nan(data):
    n = len(data)
    if n == 0:
        return data

    indices = np.arange(n)
    valid = ~np.isnan(data)

    if valid.sum() == 0:
        return np.zeros_like(data)

    return np.interp(indices, indices[valid], data[valid])

# --- Step 4: Save raw numpy arrays with NaNs interpolated ---

np_folder = 'ecg_numpy_arrays'
os.makedirs(np_folder, exist_ok=True)

for _, row in df.iterrows():
    patient_id = row['patient_id']
    ecg_array = row['data_array']
    ecg_array_clean = interpolate_nan(ecg_array)
    np.save(os.path.join(np_folder, f"{patient_id}_ecg.npy"), ecg_array_clean)

print(f"Saved {len(df)} raw ECG numpy files with NaNs interpolated.")

Saved 99 raw ECG numpy files with NaNs interpolated.


In [2]:
df.head()

,patient_id,data,label,data_array
0,284,24433.0 24317.0 24354.0 24377.0 24270.0 24272....,1,"[24433.0, 24317.0, 24354.0, 24377.0, 24270.0, ..."
1,286,-31920.0 -31920.0 -31920.0 -31920.0 -31920.0 -...,1,"[-31920.0, -31920.0, -31920.0, -31920.0, -3192..."
2,296,3221.0 3274.0 3380.0 3098.0 2843.0 2910.0 3274...,1,"[3221.0, 3274.0, 3380.0, 3098.0, 2843.0, 2910...."
3,299,18584.0 18488.0 18648.0 19008.0 19239.0 19347....,1,"[18584.0, 18488.0, 18648.0, 19008.0, 19239.0, ..."
4,303,26985.0 10038.0 -9400.0 -20411.0 -18704.0 -419...,1,"[26985.0, 10038.0, -9400.0, -20411.0, -18704.0..."


In [3]:
from scipy.signal import butter, filtfilt, resample_poly, medfilt
import numpy as np
from scipy.signal import medfilt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

In [4]:
# Butterworth low-pass filter and helpers

def butter_lowpass(cutoff, fs, order=4):
    from scipy.signal import butter
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=3):
    from scipy.signal import filtfilt
    b, a = butter_lowpass(cutoff, fs, order=order)
    data = data[:, np.newaxis] if data.ndim == 1 else data
    filtered = filtfilt(b, a, data, axis=0)
    return filtered.squeeze()

def resample_ecg(data, original_fs, target_fs):
    from scipy.signal import resample_poly
    from math import gcd
    g = gcd(int(original_fs), int(target_fs))
    up = int(target_fs // g)
    down = int(original_fs // g)
    data = data[:, np.newaxis] if data.ndim == 1 else data
    resampled = resample_poly(data, up, down, axis=0)
    return resampled.squeeze()

def median_filter_denoise(signal, kernel_size=5):
    return medfilt(signal, kernel_size=kernel_size)

In [ ]:
# Process raw ECG numpy files: filtering, resampling, median filtering

original_fs = 310.0
target_fs = 45.0
cutoff_frequency = 20.0
filter_order = 3
median_kernel_size = 5

filtered_folder = 'ecg_resampled_filtered_arrays_order3'
os.makedirs(filtered_folder, exist_ok=True)

print("Starting ECG signal processing (filter, resample, median filter)...")
for filename in tqdm(os.listdir(np_folder)):
    if filename.endswith('.npy'):
        file_path = os.path.join(np_folder, filename)
        ecg_data = np.load(file_path)

        # Resample
        ecg_resampled = resample_ecg(ecg_data, original_fs, target_fs)

        # Median filter to remove spikes
        ecg_median = median_filter_denoise(ecg_resampled, kernel_size=median_kernel_size)

        # Butterworth low-pass filter to smooth signal
        ecg_filtered = butter_lowpass_filter(ecg_median, cutoff_frequency, target_fs, filter_order)

        # Save filtered ECG
        save_path = os.path.join(filtered_folder, filename.replace('_ecg.npy', '_processed.npy'))
        np.save(save_path, ecg_filtered)

print(f"Processing completed. Saved filtered ECG files to '{filtered_folder}'.")

Starting ECG signal processing (filter, resample, median filter)...


100%|██████████| 99/99 [00:06<00:00, 15.98it/s]

Processing completed. Saved filtered ECG files to 'ecg_resampled_filtered_arrays_order3'.


In [6]:
# Define Dataset class to load processed raw ECG and segment into windows

class RawECGSegmentDataset(Dataset):
    def __init__(self, np_folder, labels_dict, fs=45, window_sec=60, overlap_sec=10):
        self.fs = fs
        self.window_sec = window_sec
        self.overlap_sec = overlap_sec
        self.samples = []
        self.labels = []

        for file in tqdm(os.listdir(np_folder), desc="Loading processed ECG files"):
            if file.endswith('.npy'):
                pid = int(file.split('_')[0])
                ecg = np.load(os.path.join(np_folder, file))

                window_len = int(window_sec * fs)
                overlap_len = int(overlap_sec * fs)
                step = window_len - overlap_len

                for start in range(0, len(ecg) - window_len + 1, step):
                    segment = ecg[start:start+window_len]
                    self.samples.append(segment)
                    self.labels.append(labels_dict[pid])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return torch.tensor(self.samples[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)


In [7]:
# Instantiate dataset and split into train/val

labels_dict = dict(zip(df['patient_id'].astype(int), df['label']))
raw_dataset = RawECGSegmentDataset(np_folder=filtered_folder, labels_dict=labels_dict,
                                  fs=target_fs, window_sec=60, overlap_sec=10)

print(f"Total segments in dataset: {len(raw_dataset)}")

Loading processed ECG files: 100%|██████████| 198/198 [00:01<00:00, 103.53it/s]

Total segments in dataset: 14006


In [8]:
# Extract features and labels arrays to split
features = [raw_dataset[i][0].numpy() for i in range(len(raw_dataset))]
labels = [raw_dataset[i][1].item() for i in range(len(raw_dataset))]

X_train, X_val, y_train, y_val = train_test_split(features, labels,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=labels)

class ECGTensorDataset(Dataset):
    def __init__(self, signals, labels):
        self.signals = torch.tensor(signals, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return len(self.signals)
    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

train_dataset = ECGTensorDataset(X_train, y_train)
val_dataset = ECGTensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


C:\Users\sajee\AppData\Local\Temp\ipykernel_19124\487114622.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  self.signals = torch.tensor(signals, dtype=torch.float32)


In [9]:
# --- Step 9: Define 1D CNN + Attention model ---

class RawECGAttention1DCNN(nn.Module):
    def __init__(self, input_len, num_classes=2, dropout=0.3):
        super(RawECGAttention1DCNN, self).__init__()

        self.conv1 = nn.Conv1d(1, 32, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)

        self.dropout = nn.Dropout(dropout)

        self.attention = nn.Sequential(
            nn.Conv1d(128, 128, kernel_size=1),
            nn.Tanh(),
            nn.Conv1d(128, 1, kernel_size=1),
            nn.Softmax(dim=2)
        )

        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))

        attn_weights = self.attention(x)
        x = (x * attn_weights).sum(dim=2)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [11]:
# Train and validate the model with AdamW and early stopping

import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

input_len = X_train[0].shape[0]
num_classes = len(set(y_train))

model = RawECGAttention1DCNN(input_len=input_len, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

num_epochs = 50
patience = 7
best_val_acc = 0.0
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    train_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = correct / total

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Accuracy={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        early_stop_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print("Saved best model")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break


Using device: cpu


Epoch 1/50 Training: 100%|██████████| 351/351 [02:48<00:00,  2.08it/s]


Epoch 1: Train Loss=0.6689, Val Accuracy=0.6463
Saved best model


Epoch 2/50 Training: 100%|██████████| 351/351 [02:35<00:00,  2.26it/s]


Epoch 2: Train Loss=0.6446, Val Accuracy=0.6574
Saved best model


...

Epoch 48: Train Loss=0.2219, Val Accuracy=0.9026


Epoch 49/50 Training: 100%|██████████| 351/351 [02:29<00:00,  2.34it/s]


Epoch 49: Train Loss=0.2112, Val Accuracy=0.9108


Epoch 50/50 Training: 100%|██████████| 351/351 [02:30<00:00,  2.34it/s]


Epoch 50: Train Loss=0.2231, Val Accuracy=0.9201
Saved best model


In [ ]:
# plt.figure(figsize=(12,5))

# plt.subplot(1,2,1)
# plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
# plt.plot(range(1, len(val_losses)+1), val_losses, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()

# plt.subplot(1,2,2)
# plt.plot(range(1, len(val_accuracies)+1), val_accuracies, label='Validation Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Validation Accuracy')
# plt.legend()

# plt.tight_layout()
# plt.show()


In [12]:
from sklearn.metrics import classification_report

# Make sure your model and device are defined as in training
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

class_names = ['Poor', 'Good']  # Update as per your classes

report = classification_report(all_labels, all_preds, target_names=class_names, digits=2)
print("Detailed Classification Report on Validation Set:")
print(report)


Detailed Classification Report on Validation Set:
              precision    recall  f1-score   support

        Poor       0.93      0.91      0.92      1442
        Good       0.91      0.93      0.92      1360

    accuracy                           0.92      2802
   macro avg       0.92      0.92      0.92      2802
weighted avg       0.92      0.92      0.92      2802



In [13]:
# Load best model and print classification report

from sklearn.metrics import classification_report

# After model evaluation and collecting all_preds and all_labels

class_names = ['Poor', 'Good']  # Replace with your actual class names

report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
print("Detailed classification report on validation set:")
print(report)


Detailed classification report on validation set:
              precision    recall  f1-score   support

        Poor     0.9307    0.9126    0.9216      1442
        Good     0.9092    0.9279    0.9185      1360

    accuracy                         0.9201      2802
   macro avg     0.9200    0.9203    0.9200      2802
weighted avg     0.9203    0.9201    0.9201      2802



In [ ]:
# Steps 1–4: Your original code to load and save raw ECG arrays.

# Steps 5–6: Filter, resample, median-filter, and save processed ECG arrays.

# Step 7: New Dataset that loads these processed ECG signals and segments into smaller windows.

# Step 8: Split dataset and create PyTorch DataLoader.

# Step 9: Define the 1D CNN + attention model that takes raw ECG windows.

# Step 10: Train with AdamW optimizer, CrossEntropyLoss, early stopping.

# Step 11: Evaluate best model and print detailed classification report.

In [ ]:
# Input: Takes a raw ECG signal (a sequence of numbers representing heart activity over time).

# Pre-process: median and butterworth-low pass filter

# CNN Layers:

# The input raw ECG window (a sequence of voltage values over time) passes through several 1D convolutional layers.

# Each convolution layer extracts increasingly complex features:

# Early layers might detect simple patterns like edges or peaks (e.g., QRS complexes).

# Deeper layers combine those to capture rhythms, morphology, and higher-level ECG patterns.

# These layers output a feature map — a set of learned features across time and channels.

# Attention Layer:

# The attention mechanism takes these CNN feature maps as input.

# It assigns weights (importance scores) to each time step or segment of the feature map.

# This means it highlights which parts of the signal are most relevant for classification.

# It produces a weighted summary (a context vector), emphasizing important time regions dynamically.

# Classification: Passes the summarized features through two dense layers to predict the class (e.g., normal or abnormal heart activity).

# Regularization: Uses dropout to prevent overfitting and batch normalization to stabilize training.